# Лабораторная работа №3

## Задание 1 . 
###### Провести классификацию найденного датасета, методами линеной и логистической регрессий . В формате Markdown написать пояснения. Объяснить почему были выбраны именно такие гиперпараметры, была ли перекрестная проверка, и т.д.

датасет: [Homicide Reports, 1980-2014](https://www.kaggle.com/datasets/murderaccountability/homicide-reports)

### Загрузка Библиотек

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler

### Загрузка данных и предварительная обработка
###### Загрузка данных

In [2]:
N      = 200000
N_test = 0.99992

In [3]:
data = pd.read_csv('database.csv',low_memory=False)
subset_data = data.sample(n=N, random_state=42)
# Исключение ненужных столбцов
columns_to_drop = ['Record ID', 'Agency Code', 'Agency Name', 'Agency Type', 'City', 'State', 'Year', 'Month', 'Record Source']
subset_data = subset_data.drop(columns=columns_to_drop)

subset_data.dropna(inplace=True) # Удаление записей с пропущенными значениями
subset_data.head()

,Incident,Crime Type,Crime Solved,Victim Sex,Victim Age,Victim Race,Victim Ethnicity,Perpetrator Sex,Perpetrator Age,Perpetrator Race,Perpetrator Ethnicity,Relationship,Weapon,Victim Count,Perpetrator Count
484984,1,Murder or Manslaughter,No,Male,18,White,Hispanic,Unknown,0,Unknown,Unknown,Unknown,Handgun,1,0
608445,3,Murder or Manslaughter,No,Male,27,White,Not Hispanic,Unknown,0,Unknown,Unknown,Unknown,Firearm,0,0
417612,1,Murder or Manslaughter,Yes,Male,42,Black,Not Hispanic,Male,18,Black,Not Hispanic,Acquaintance,Handgun,0,0
372692,6,Murder or Manslaughter,No,Male,44,White,Not Hispanic,Unknown,0,Unknown,Unknown,Unknown,Handgun,0,0
402028,1,Murder or Manslaughter,Yes,Female,15,White,Unknown,Male,20,White,Unknown,Girlfriend,Rifle,0,0


In [4]:
# Выбор признаков и целевой переменной
X = subset_data.drop(columns=['Crime Solved'])
#y = subset_data['Crime Solved']
y = subset_data['Crime Solved'].apply(lambda x: 1 if x == 'Yes' else 0)  # Преобразование целевой переменной в бинарный формат
X = pd.get_dummies(X)

# Масштабирование данных
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

### Разделение данных на тренировочную и тестовую выборки

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=N_test, random_state=42)

### Построение и оценка модели

### Линейная регрессия

In [6]:
%%time
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

# Предсказания и оценка модели
y_pred_lin = lin_reg.predict(X_test)
y_pred_lin_class = [1 if pred >= 0.5 else 0 for pred in y_pred_lin]

print("Линейная регрессия:")
print(classification_report(y_test, y_pred_lin_class))
print("Матрица ошибок:\n", confusion_matrix(y_test, y_pred_lin_class))
print("Точность:", accuracy_score(y_test, y_pred_lin_class))

Линейная регрессия:
              precision    recall  f1-score   support

           0       0.74      1.00      0.85     59663
           1       1.00      0.85      0.92    140321

    accuracy                           0.90    199984
   macro avg       0.87      0.93      0.89    199984
weighted avg       0.92      0.90      0.90    199984

Матрица ошибок:
 [[ 59647     16]
 [ 20663 119658]]
Точность: 0.8965967277382191
CPU times: total: 1.42 s
Wall time: 1.14 s


### Логистическая регрессия

In [7]:
%%time
log_reg = LogisticRegression(max_iter=5000)

# Определение гиперпараметров для GridSearchCV
param_grid = {
    'penalty': ['l1', 'l2', 'elasticnet'],
    'C': [0.1, 1, 10, 100],
    'solver': ['lbfgs', 'saga', 'liblinear']
}

# Поиск лучших гиперпараметров с помощью GridSearchCV
grid_search = GridSearchCV(log_reg, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)
print("Лучшие гиперпараметры логистической регрессии:", grid_search.best_params_)

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Лучшие гиперпараметры логистической регрессии: {'C': 0.1, 'penalty': 'l2', 'solver': 'liblinear'}
CPU times: total: 4.34 s
Wall time: 4.48 s


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
80 fits failed out of a total of 180.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py", line 1151, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\ProgramData\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py", line 1168, in fit
    solver = 

In [8]:
%%time
# Оценка модели с лучшими гиперпараметрами
best_log_reg = grid_search.best_estimator_
y_pred_log = best_log_reg.predict(X_test)

print("Логистическая регрессия:")
print(classification_report(y_test, y_pred_log))
print("Матрица ошибок:\n", confusion_matrix(y_test, y_pred_log))
print("Точность:", accuracy_score(y_test, y_pred_log))

Логистическая регрессия:
              precision    recall  f1-score   support

           0       0.79      1.00      0.88     59663
           1       1.00      0.88      0.94    140321

    accuracy                           0.92    199984
   macro avg       0.89      0.94      0.91    199984
weighted avg       0.94      0.92      0.92    199984

Матрица ошибок:
 [[ 59628     35]
 [ 16204 124117]]
Точность: 0.9187985038803105
CPU times: total: 1.08 s
Wall time: 918 ms
